In [1]:
import numpy as np
import cv2 as cv
import time

In [2]:
source = 0
show_hsv = False
show_glitch = False

In [3]:
def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = np.zeros(img.shape)+255
    # vis = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
    cv.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

In [4]:
def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    return bgr

def warp_flow(img, flow):
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv.remap(img, flow, None, cv.INTER_LINEAR)
    return res

In [5]:
cv.namedWindow("flow")

cam = cv.VideoCapture(source)

[ WARN:0@0.358] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [6]:
_ret, prev_frame = cam.read()

prev_frame_gray = cv.cvtColor(prev_frame, cv.COLOR_BGR2GRAY)
cur_glitch = prev_frame.copy()

while _ret:
    _ret, frame = cam.read()
    
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(
        prev_frame_gray,
        frame_gray,
        flow=None,
        pyr_scale=0.5,
        levels=3,
        winsize=15,
        iterations=3,
        poly_n=5,
        poly_sigma=1.2,
        flags=0
    )
    prev_frame_gray = frame_gray

    cv.imshow('flow', draw_flow(frame_gray, flow))
    if show_hsv:
        cv.imshow('flow HSV', draw_hsv(flow))
    if show_glitch:
        cur_glitch = warp_flow(cur_glitch, flow)
        cv.imshow('glitch', cur_glitch)

    key = cv.waitKey(5)
    if key == 27: # exit on ESC
        break
    if key == ord('1'):
        show_hsv = not show_hsv
        print('HSV flow visualization is', ['off', 'on'][show_hsv])
    if key == ord('2'):
        show_glitch = not show_glitch
        if show_glitch:
            cur_glitch = frame.copy()
        print('glitch is', ['off', 'on'][show_glitch])

KeyboardInterrupt: 

In [ ]:
cv.destroyAllWindows()
cv.waitKey(1)